In [2]:
import numpy as np
import soundfile as sf

# Define Fractional Delay Filter
def fractional_delay(input_signal, delay):
    output_signal = np.zeros_like(input_signal)
    for i in range(len(input_signal)):
        index = i - delay
        index_floor = int(np.floor(index))
        frac = index - index_floor
        if index_floor < 0 or index_floor >= len(input_signal) - 1:
            continue
        output_signal[i] = (1 - frac) * input_signal[index_floor] + frac * input_signal[index_floor + 1]
    return output_signal

# Allpass Filter using Fractional Delay
def fractional_allpass(input_signal, delay, gain):
    delayed_signal = fractional_delay(input_signal, delay)
    return -gain * input_signal + delayed_signal + gain * delayed_signal[0]

# Comb Filter using Fractional Delay
def fractional_comb(input_signal, delay, gain):
    delayed_signal = fractional_delay(input_signal, delay)
    return input_signal + gain * delayed_signal

# Freeverb using Fractional Delay
def fractional_freeverb(input_signal, fs, rt60):
    # Define delay lengths
    allpass_delays = [347.7, 113.1, 37.8]  # in samples
    comb_delays = [1687.3, 1601.5, 2053.7, 2251.1]  # in samples

    # Calculate decay factors for 60dB decay
    allpass_gains = [0.7, 0.7, 0.7]
    comb_gains = [10 ** (-3 * d / (fs * rt60)) for d in comb_delays]

    # Apply allpass filters
    for delay, gain in zip(allpass_delays, allpass_gains):
        input_signal = allpass(input_signal, delay, gain)

    # Apply comb filters
    output_signal = np.zeros_like(input_signal)
    for delay, gain in zip(comb_delays, comb_gains):
        output_signal += comb(input_signal, delay, gain)

    return output_signal

# Load an audio file
input_signal, fs = sf.read('data/freeverb_dataset/dry/balloon_burst_1.wav')

# Apply Schroeder Reverb
output_signal = schroeder_reverb(input_signal, fs, rt60=1.2)  # rt60 is the reverb time in seconds

# Save the output audio file
sf.write('output.wav', output_signal, fs)